In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
df = pd.read_csv('mfcc.csv')

In [3]:
df1 = df.drop(['Path'],axis=1)

In [4]:
df1['Features'] = df1['Features'].apply(ast.literal_eval)

In [5]:
dataset = pd.DataFrame(index=df1.index, columns=[f'Feature{i+1}' for i in range(100)])
for i in range(100):
    dataset[f'Feature{i+1}'] = df1['Features'].apply(lambda x: x[i] if i < len(x) else None)
dataset['class'] = df1['Class']

/tmp/ipykernel_7832/4105123556.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset['class'] = df1['Class']


In [6]:
clss = dataset['class'].unique()
n = dataset['class'].nunique()
value = [i for i in range (0,n)]
dataset['class'] = dataset['class'].replace(clss,value)

In [7]:
dataset.to_csv('Feature.csv')

dataset.head()

## Making Speeker identification Using Descition Tree Classifier

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = dataset.iloc[:, :-1]  
y = dataset['class']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Descition = DecisionTreeClassifier()

Descition.fit(X_train, y_train)

y_pred = Descition.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.8215042372881356


In [9]:
import librosa
import librosa.display
file = 'SPEECH_DATA/FEMALE/MIC/F02/mic_F02_si646.wav'
def features_extractor(file):
    #load the file (audio)
    audio, sample_rate = librosa.load(file) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=100)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features


k = Descition.predict([features_extractor(file)]).tolist()
clss[k]

/home/aldrax/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['FEMALE-MIC-F02'], dtype=object)

In [10]:
%%time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error



X = dataset.iloc[:, :-1].values  
y = dataset['class'].values  
input_size = 100
output_size = 40
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X_train.shape[1]
output_size = len(np.unique(y_train))
model = SimpleNN(input_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.numpy())
            all_labels.extend(labels.numpy())

        accuracy = correct / total
        mae = mean_absolute_error(all_labels, all_preds)

print(f'Epoch [{epoch + 1}/{num_epochs}], Accuracy: {accuracy:.4f}, MAE: {mae:.4f}')
torch.save(model.state_dict(), 'model/model.pth')

Epoch [20/20], Accuracy: 0.9582, MAE: 0.1621
CPU times: user 10.8 s, sys: 211 ms, total: 11.1 s
Wall time: 3.92 s


In [11]:
import librosa
import librosa.display
model = SimpleNN(100,40)
model.load_state_dict(torch.load('model/model.pth'))
model.eval()

def features_extractor(file):
    #load the file (audio)
    audio, sample_rate = librosa.load(file) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=100)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

file = 'SPEECH_DATA/MALE/MIC/M02/mic_M02_sa2.wav'
input_data = torch.from_numpy(features_extractor(file))
# Performing a forward pass
output = model(input_data)
index = torch.argmax(output).item()

# print("Model Output:",max(output))
print("Model Output", clss[index])


Model Output MALE-MIC-M02


# Task 3
This task add random noise to our dataset and after adding noise we will try to classify the audio with added noise

Task 3: Now add noise to your data and again perform the speaker identification. You can use audio augmentation so improve the accuracy. Train the model with noisy data.

In [12]:
def add_noise(audio, noise_level=0.005):
    audio, sample_rate = librosa.load(file) 
    noise = np.random.randn(len(audio)) * noise_level
    noisy_audio = audio + noise
    noisy_audio = np.clip(noisy_audio, -1.0, 1.0)    
    return noisy_audio , sample_rate

def features_extractor(audio , sample_rate):
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=100)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features
file = 'SPEECH_DATA/MALE/MIC/M02/mic_M02_sa2.wav'
data = add_noise(file)

In [13]:
from IPython.display import Audio
Audio(file)

In [14]:
audio, sample_rate = librosa.load(file) 
Audio(data[0], rate=sample_rate)

In [15]:
%%time
import os
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
res = []
clss = []
mfcc = []
count = 0
data_folder = 'SPEECH_DATA'
for path in os.listdir(data_folder):
    if os.path.isfile(os.path.join(data_folder, path)):
        res.append(path)
        clss.append(path)
    else:
        for i in os.listdir(data_folder+'/'+path):
            if os.path.isfile(os.path.join(data_folder+path, i)):
                res.append(os.path.join(data_folder+'/'+path,i))
                clss.append(path+'-'+i)
            else:
                for j in os.listdir(data_folder+'/'+path+'/'+i):
                    if os.path.isfile(os.path.join(data_folder+'/'+path+'/'+i,j)):
                        try:
                            res.append(os.path.join(data_folder+'/'+path+'/'+i,j))
                            clss.append(path+'-'+i)
                            sample_rate, audio_data = wavfile.read(os.path.join(data_folder+'/'+path+'/'+i,j))
                            count += 1
                            print(count,end='/')
                        except ValueError as ve:
                            print(f"Error during spectrogram computation: {ve}")
                    for k in os.listdir(data_folder+'/'+path+'/'+i+'/'+j):
                        if os.path.isfile(os.path.join(data_folder+'/'+path+'/'+i+'/'+j,k)):
                            try:
                                val = os.path.join(data_folder+'/'+path+'/'+i+'/'+j,k)
                                c = path+'-'+i+'-'+j
                                res.append(val)
                                clss.append(c)
                                noise ,sr = add_noise(val)
                                Features = features_extractor(noise,sr).tolist()
                                mfcc.append(Features)
                                count += 1
                            except Exception as e:
                                print(f'An unexpected error occurred:,{e}', end = '')
                                mfcc.append(None)                                
df = pd.DataFrame({'Path':res,'Class':clss,'Features':mfcc})
df.head()
print(count)

14154
CPU times: user 13min 22s, sys: 9.51 s, total: 13min 31s
Wall time: 3min 23s


In [16]:
df1 = df.dropna()
df1.to_csv('added_noise.csv',index = False)
df1 = pd.read_csv('added_noise.csv')
df1.head()

,Path,Class,Features
0,SPEECH_DATA/FEMALE/LAR/F04/lar_F04_sx155.wav,FEMALE-LAR-F04,"[-279.12816186913, 23.591633053802695, 5.00308..."
1,SPEECH_DATA/FEMALE/LAR/F04/lar_F04_si1183.wav,FEMALE-LAR-F04,"[-279.684073066372, 23.392510005767893, 5.2808..."
2,SPEECH_DATA/FEMALE/LAR/F04/lar_F04_si1062.wav,FEMALE-LAR-F04,"[-279.3070185405566, 23.494615070675973, 4.708..."
3,SPEECH_DATA/FEMALE/LAR/F04/lar_F04_si1184.wav,FEMALE-LAR-F04,"[-279.20103284598537, 23.950137924647564, 5.22..."
4,SPEECH_DATA/FEMALE/LAR/F04/lar_F04_sx150.wav,FEMALE-LAR-F04,"[-279.59607885572774, 23.51247941286268, 5.364..."


In [17]:
df1['Features'] = df1['Features'].apply(ast.literal_eval)
dataset = pd.DataFrame(index=df1.index, columns=[f'Feature{i+1}' for i in range(100)])
for i in range(100):
    dataset[f'Feature{i+1}'] = df1['Features'].apply(lambda x: x[i] if i < len(x) else None)
dataset['class'] = df1['Class']

/tmp/ipykernel_7832/3717272120.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset['class'] = df1['Class']


In [18]:
clss = dataset['class'].unique()
n = dataset['class'].nunique()
value = [i for i in range (0,n)]
dataset['class'] = dataset['class'].replace(clss,value)

### Using Descition Tree for  Classification

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = dataset.iloc[:, :-1]  
y = dataset['class']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Descition = DecisionTreeClassifier()
Descition.fit(X_train, y_train)
y_pred = Descition.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

## model failed due to adding noise

Accuracy: 0.017661603673613566


In [20]:
file = 'SPEECH_DATA/FEMALE/MIC/F02/mic_F02_si646.wav'
audio , sr = librosa.load(file)

# classifying without adding Noise 
audio,sr = add_noise(file)
k = Descition.predict([features_extractor(audio , sr)]).tolist()
clss[k]

/home/aldrax/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['MALE-MIC-M10'], dtype=object)

In [24]:
%%time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

input_size = 100
output_size = 60

X = dataset.iloc[:, :-1].values  
y = dataset['class'].values  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X_train.shape[1]
output_size = len(np.unique(y_train))
model = SimpleNN(input_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.numpy())
            all_labels.extend(labels.numpy())

        accuracy = correct / total
        mae = mean_absolute_error(all_labels, all_preds)

print(f'Epoch [{epoch + 1}/{num_epochs}], Accuracy: {accuracy:.4f}, MAE: {mae:.4f}')
torch.save(model.state_dict(), 'model/Noise_model.pth')

Epoch [30/30], Accuracy: 0.0166, MAE: 15.0307
CPU times: user 20.6 s, sys: 208 ms, total: 20.8 s
Wall time: 5.22 s


In [25]:
import librosa
import librosa.display
model = SimpleNN(100,60)
model.load_state_dict(torch.load('model/Noise_model.pth'))
model.eval()

def features_extractor(file):
    #load the file (audio)
    audio, sample_rate = librosa.load(file) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=100)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

file = 'SPEECH_DATA/MALE/MIC/M02/mic_M02_sa2.wav'
input_data = torch.from_numpy(features_extractor(file))
# Performing a forward pass
output = model(input_data)
index = torch.argmax(output).item()

# print("Model Output:",max(output))
print("Model Output", clss[index])

Model Output MALE-LAR-M05
